In [6]:
import pandas as pd
import os

# Ruta de la carpeta que contiene los archivos TSV
carpeta = "C:/Users/Lenovo/Desktop/tesis/repositorio/precipitaciones/data"

# Lista para almacenar los DataFrames
dfs = []

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    # Construir la ruta completa del archivo
    ruta_archivo = os.path.join(carpeta, archivo)
    
    # Verificar si el archivo es un archivo TSV
    if archivo.endswith('.tsv'):
        # Leer el archivo y agregarlo a la lista de DataFrames
        try:
            df = pd.read_csv(ruta_archivo, sep='\t')  # Asumiendo que el separador es una tabulación
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)

# Mostrar las primeras filas del DataFrame final
print(df_final.head())


  reservorio       fecha agua_caida
0  ACONCAGUA  2021-01-01        0,0
1      CHAPO  2021-01-01        0,0
2     COLBÚN  2021-01-01        0,0
3  INVERNADA  2021-01-01        0,0
4       LAJA  2021-01-01        0,0


In [7]:
df_final

,reservorio,fecha,agua_caida
0,ACONCAGUA,2021-01-01,"0,0"
1,CHAPO,2021-01-01,"0,0"
2,COLBÚN,2021-01-01,"0,0"
3,INVERNADA,2021-01-01,"0,0"
4,LAJA,2021-01-01,"0,0"
...,...,...,...
37109,PANGUE,2020-12-31,"0,0"
37110,PILMAIQUEN,2020-12-31,"0,0"
37111,PULLINQUE,2020-12-31,"0,0"
37112,RAPEL,2020-12-31,"0,0"


In [8]:
# Reemplazando comas por puntos en la columna 'agua_caida' y convirtiendo a float
df_final['agua_caida'] = df_final['agua_caida'].str.replace(',', '.').astype(float)

# Convirtiendo la columna 'fecha' a datetime si aún no se ha hecho
df_final['fecha'] = pd.to_datetime(df_final['fecha'])

# Ordenando el DataFrame por la columna 'fecha'
df_final = df_final.sort_values(by='fecha')

# Mostrando los tipos de datos y las primeras filas del DataFrame corregido
print(df_final.dtypes)
print(df_final.head())

reservorio            object
fecha         datetime64[ns]
agua_caida           float64
dtype: object
      reservorio      fecha  agua_caida
13341  ACONCAGUA 2016-01-01         0.0
13342      CHAPO 2016-01-01         0.0
13343     COLBÚN 2016-01-01         0.0
13344  INVERNADA 2016-01-01         0.0
13345       LAJA 2016-01-01         0.0


In [10]:
# Obteniendo la lista de valores únicos en la columna 'reservorio'
valores_unicos_reservorio = df_final['reservorio'].unique()

# Imprimiendo los valores únicos
print(valores_unicos_reservorio)

['ACONCAGUA' 'CHAPO' 'COLBÚN' 'INVERNADA' 'LAJA' 'LOS MOLLES' 'MELADO'
 'PANGUE' 'PILMAIQUEN' 'PULLINQUE' 'RAPEL' 'SAUZAL' 'LA MINA' 'PEHUENCHE'
 'CIPRESES' 'ABANICO']


In [11]:
df_sur = df_final[df_final['reservorio'] == 'CHAPO'][['fecha', 'agua_caida']].rename(columns={'agua_caida': 'lluvia_zona_sur'})
df_centro_sur = df_final[df_final['reservorio'].isin(['INVERNADA', 'PEHUENCHE', 'CIPRESES'])].groupby('fecha')['agua_caida'].mean().reset_index().rename(columns={'agua_caida': 'lluvia_zona_centro_sur'})
df_centro = df_final[df_final['reservorio'].isin(['LAJA', 'RAPEL', 'ABANICO', 'PANGUE'])].groupby('fecha')['agua_caida'].mean().reset_index().rename(columns={'agua_caida': 'lluvia_zona_centro'})

# Uniendo los DataFrames
df_lluvias_zonas = pd.merge(df_sur, df_centro_sur, on='fecha', how='outer')
df_lluvias_zonas = pd.merge(df_lluvias_zonas, df_centro, on='fecha', how='outer')

# Ordenando por fecha
df_lluvias_zonas = df_lluvias_zonas.sort_values(by='fecha')

# Mostrando las primeras filas del nuevo DataFrame
print(df_lluvias_zonas.head())

       fecha  lluvia_zona_sur  lluvia_zona_centro_sur  lluvia_zona_centro
0 2016-01-01              0.0                     0.0                 0.0
1 2016-01-02              0.0                     0.0                 0.0
2 2016-01-03              0.0                     0.0                 0.0
3 2016-01-04              0.0                     0.0                 0.0
4 2016-01-05              0.0                     0.0                 0.0


In [15]:
import plotly.express as px

# Asegúrate de que df_lluvias_zonas es tu DataFrame

# Creando el gráfico de líneas con Plotly
fig = px.line(df_lluvias_zonas, x='fecha', y=['lluvia_zona_sur', 'lluvia_zona_centro_sur', 'lluvia_zona_centro'],
              title='Series de Tiempo de Precipitaciones por Zona',
              labels={'value': 'Precipitaciones (mm)', 'variable': 'Zonas'},
              markers=False)

# Configuración de la leyenda y títulos de los ejes
fig.update_layout(legend_title_text='Zonas',
                  xaxis_title='Fecha',
                  yaxis_title='Precipitaciones (mm)')

# Mostrando el gráfico
fig.show()
